In [1]:
#auto update selenium
# webdriver_auto_update.py
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time 
import os

print("Checking webdriver")
#check and install Selenium web driver.
chromedriver_autoinstaller.install()

Checking webdriver


'/Users/aaronbethea/Documents/SoftwareDev/pythonApps/cfvg_od-image_scraper/cfvg_env/lib/python3.10/site-packages/chromedriver_autoinstaller/100/chromedriver'

In [15]:
def get_selenium():                           
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('headless')
    options.add_argument('log-level=2') #display erros only                  
    driver = webdriver.Chrome(options=options)
    return (driver)

In [16]:
#scrape cards from CFVG site
#resource_url = "https://en.cf-vanguard.com/cardlist/cardsearch/?expansion=167"

#pull from the card-detail view
resource_url = "https://en.cf-vanguard.com/cardlist/cardsearch/?expansion=167&view=text&sort=no"

In [18]:
#load th browser object
browser = get_selenium()                           



KeyboardInterrupt: 

In [19]:
#get length of page
browser.get(resource_url)
print("Get page length")
lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight); var lenOfPage=document.body.scrollHeight;return lenOfPage;")
print(lenOfPage)

Start page load & Scroll
6571


In [21]:
# Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
from bs4 import BeautifulSoup

match = False
print("Begin Page")
scrolls = 0
while not match:
    last_count = lenOfPage
    time.sleep(5)
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    scrolls += 1
    if last_count == lenOfPage:
        match = True
print(f"Total Scrolls: {scrolls}")
#load page data
print("Get source data")
source_data = browser.page_source

#grab beautiful soup object
page_soup = BeautifulSoup(source_data)
print("data loaded")




Begin Page
Total Scrolls: 14
Get source data
data loaded


In [24]:
results = page_soup.findAll("div","cardlist_imagelist")[0]
cards = results.findAll("li")
print(len(cards))
sample = cards[0]

213


In [77]:
def get_card_data(element):
    '''
    @param element BS4 list element
    '''
    card_data = {}

    text_data = element.find("div","text")
    stats_data = str(text_data.find("div","status").text).strip().replace("｜","").split("\n")
    #indexes
    #card_type
    #nation
    #grade
    #power
    #shield

    card_data.update({
        "card_num":str(text_data.find('div','number').text).strip(),
        "card_name":str(text_data.find('h5').text).strip(),
        "card_type":stats_data[0],
        "nation":stats_data[1], 
        "grade":stats_data[2].split(" ")[-1],
        "power":stats_data[3].split(" ")[-1],
        "shield":stats_data[-1].split(" ")[-1],
        "image":element.find("div","image").img.get('src'),
        "text":text_data.find("p").text,
    })
    return card_data

In [79]:
db = []
print("Parsing Card Data")
for c in cards:
    db.append(get_card_data(c))
print("Complete")

Parsing Card Data
Complete


In [65]:
# print(sample.a.get("href")) #bushi backend url
# img_data = sample.find("div","image")
# print(f"Image Link: {img_data.img.get('src')}")
# #text
# text_data = sample.find("div","text")
# print(f"Card Number: {str(text_data.find('div','number').text).strip()}")
# print(f"Card Name: {str(text_data.find('h5').text).strip()}")
# print("Card Stats")
# stats_data = str(text_data.find("div","status").text).strip().replace("｜","").split("\n")
# print(stats_data)
# #card text
# print(text_data.find("p").text)
# #print("data")
# #print(text_data)

/cardlist/?cardno=D-BT01/001EN&expansion=167&view=text
Image Link: https://s3-ap-northeast-1.amazonaws.com/en.cf-vanguard.com/wordpress/wp-content/images/cardlist/dbt01/dbt01_001.png
Card Number: D-BT01/001EN
Card Name: Vairina Valiente
Card Stats
['Normal Unit', 'Dragon Empire', 'Grade 3', 'Power 13000', 'Shield -']
[overDress]－"Trickstar", or a grade 3 or less unit in the [overDress] state (You may place it by stacking it on the specified unit instead of normal calling it to (RC))[CONT](RC):This unit gets [Power] +5000 for each of this unit's originalDress. [AUTO](RC)[1/turn]:When the attack of this unit in the [overDress] state hits, [COST][Counter-Blast]1 & discard a card from your hand], and [Stand] this unit.


In [81]:
#save the list as a json obj
import json 
print("Writing file")
with open("dbt01.json","w") as jf:
    jf.write(json.dumps(db,indent=4))
print("Done")

Writing file
Done
